In [1]:
import os

In [3]:
import pickle
from IPython import get_ipython


def run_abcrown_from_dict_ipython(args_dict, debug_file):
    cmd = "python alpha-beta-CROWN/complete_verifier/abcrown.py"

    output_path = args_dict["output_file"]
    results_path = args_dict["results_file"]

    if os.path.isfile(output_path):
        os.remove(output_path)
    if os.path.isfile(results_path):
        os.remove(results_path)
    
    # Append each argument with "--" prefix
    for key, value in args_dict.items():
        cmd += f" --{key} {value}"

    # cmd = f"{cmd} > {debug_file}"

    get_ipython().system(cmd)



In [ ]:
model_path = 'fyp3/experiments/robust_model/seed=33,epochs=3,lambda_entropy=0.025,model=CNN,learning_rate=0.001,rebalance_freq=5,jitter=1.5,loss=CROWN-IBP,alpha=0.5,robust_eps=0.05,warmup=20,checkpoint_freq=10,checkpoint=3.pth'

In [6]:
import pandas as pd

In [11]:
default_params = {
    "config": "jamie.yaml",
    "device": "cuda",
    "start": "0",
    "end": "2",
    "conv_mode": "matrix",
    "load_model": model_path,

}

results = {}#20

force_run = True

test_eps = [0.005, 0.01, 0.02, 0.05, 0.1]
test_delta = [0.05, 0.1, 0.2, 0.3]

test_eps = [0.01]
test_delta = [0.1]


for eps in test_eps:
    results[eps] = {}
    for delta in test_delta:
        path = model_path[:-4] + f"_e={eps}_d={delta}"
        output_path = path + "_output.pkl"
        results_path = path + "_results.txt" 
        debug_path = path + "_debug.txt"  

        print(debug_path)

        if not os.path.isfile(output_path) or not os.path.isfile(results_path) or force_run:
            print(f"No results found, running at {eps}, {delta}")
            args = default_params | {
                "epsilon": eps,
                "delta": delta,
                "output_file": output_path,
                "results_file": results_path,
            }
            run_abcrown_from_dict_ipython(args, debug_path)

        
        if os.path.isfile(output_path) and os.path.isfile(results_path):
            print("loading results")
            with open(output_path, "rb") as f:
                output_data = pickle.load(f)
            with open(results_path, "rb") as f:
                results_data = pickle.load(f)   
        else:
            print("failed to load results")
            output_data = None
            results_data = None    

        results[eps][delta] = {"results_data": results_data, "output_data": output_data}

        to_df = []
        for eps, eps_dict in results.items():
            for delta, delta_dict in eps_dict.items():
                summary_agg = {}
                if delta_dict["results_data"] is not None:
                    summary = delta_dict["results_data"]["summary"]
                    for k, v in summary.items():
                        summary_agg[k] = len(v)

                to_df.append({
                    "eps": eps,
                    "delta": delta,
                } | summary_agg)

        display(pd.DataFrame(to_df))


fyp3/experiments/robust_model/seed=2,epochs=75,lambda_entropy=0.025,model=CNN,learning_rate=0.001,checkpoint_freq=10,rebalance_freq=5,jitter=0.03,checkpoint=40_e=0.01_d=0.1_debug.txt
No results found, running at 0.01, 0.1
Configurations:

general:
  device: cuda
  seed: 100
  conv_mode: matrix
  deterministic: false
  double_fp: false
  loss_reduction_func: sum
  sparse_alpha: true
  sparse_interm: true
  save_adv_example: true
  verify_onnxruntime_output: false
  eval_adv_example: false
  show_adv_example: true
  precompile_jit: false
  prepare_only: false
  complete_verifier: skip
  enable_incomplete_verification: true
  csv_name: null
  results_file: 'fyp3/experiments/robust_model/seed=2,epochs=75,lambda_entropy=0.025,model=CNN,learning_rate=0.001,checkpoint_freq=10,rebalance_freq=5,jitter=0.03,checkpoint=40_e=0.01_d=0.1_results.txt'
  root_path: ''
  deterministic_opt: false
  graph_optimizer: 'Customized("custom_graph_optimizer", "default_optimizer")'
  buffer_has_batchdim: false


,eps,delta
0,0.01,0.1
